In [10]:
import os
import numpy as np
import pandas as pd
import uproot
import matplotlib.pyplot as plt
import awkward as ak

In [106]:

path = "/Users/markmatthewman/Projects/Patatrack15/data/SamplesV2/CloseByPion/histo"

v_cp_energy = []
v_tkx_energy = []
v_cp_tkx_energy_frac = []
v_tkx_numtkx = []
v_weighted_bar_x = []
v_weighted_bar_y = []
v_weighted_bar_z = []
v_cee_120 = []
v_cee_200 = []
v_cee_300 = []
v_ceh_120 = []
v_ceh_200 = []
v_ceh_300 = []
v_ceh_scint = []

def applyFilters(features,idx):
    for feature in features:
        feature
    return features


for root, dirs, files in os.walk(path):
    for file in files:
        if ".root" in file:
            f = uproot.open(os.path.join(path,file))

            # Energy Measures
            tkx_energy = np.array([ak.sum(x) for x in f["ticlDumper/trackstersMerged"]["raw_energy"].array()])
            cp_energy = np.array([ak.sum(x) for x in f["ticlDumper/simtrackstersCP"]["regressed_energy"].array()])
            cp_tkx_energy_frac = energy/cp_energy
            
            # Number of Tracksters
            tkx_numtkx =np.array([ak.count(x) for x in f["ticlDumper/trackstersMerged"]["raw_energy"].array()])

            # Barycenter
            bar_x = f["ticlDumper/trackstersMerged"]["barycenter_x"].array(library="np")
            bar_y = f["ticlDumper/trackstersMerged"]["barycenter_y"].array(library="np")
            bar_z = f["ticlDumper/trackstersMerged"]["barycenter_z"].array(library="np")
            weights = [np.array(x)/np.sum(np.array(x)) for x in f["ticlDumper/trackstersMerged"]["raw_energy"].array()]

            weighted_bar_x = np.array([np.sum([x*l]) for x, l in zip(weights, bar_x)])
            weighted_bar_y = np.array([np.sum([x*l]) for x, l in zip(weights, bar_y)])
            weighted_bar_z = np.array([np.sum([x*l]) for x, l in zip(weights, bar_z)])

            # Split by Cell Type
            cell_types = [np.array(ak.sum(x,axis=0))/l for x, l in zip(f["ticlDumper/trackstersMerged"]["raw_energy_perCellType"].array(), tkx_energy)]

            # Filter Events
            f1_idx = np.where(tkx_energy==0)    # Exclude Tracksters with no energies
            f2_idx = np.where(cp_energy<10)        # Exclude CPs with energies less than 10 GeV
            f_idx = np.union1d(f1_idx,f2_idx)   
            

            # Write quantities

            v_cp_energy.append(np.delete(cp_e, f_idx))
            v_tkx_energy.append(np.delete(tkx_energy,f_idx))
            v_cp_tkx_energy_frac.append(np.delete(tkx_energy/cp_energy,f_idx))
            v_tkx_numtkx.append(np.delete(tkx_numtkx,f_idx))
            v_weighted_bar_x.append(np.delete(weighted_bar_x,f_idx))
            v_weighted_bar_y.append(np.delete(weighted_bar_y,f_idx))
            v_weighted_bar_z.append(np.delete(weighted_bar_z,f_idx))
            
            # Split Energy per Cell
            cell_types = [i for j, i in enumerate(cell_types) if j not in f_idx]
            cell_types = np.array(cell_types)
            v_cee_120.append(cell_types[:,0])
            v_cee_200.append(cell_types[:,1])
            v_cee_300.append(cell_types[:,2])
            v_ceh_120.append(cell_types[:,3])
            v_ceh_200.append(cell_types[:,4])
            v_ceh_300.append(cell_types[:,5])
            v_ceh_scint.append(cell_types[:,6])

    
        

In [97]:
cell_types = [np.array(ak.sum(x,axis=0)) for x in f["ticlDumper/trackstersMerged"]["raw_energy_perCellType"].array()]


In [99]:
for i,ele in enumerate(cell_types):
    if len(ele)!=7:
        print(i,ele)

37 []
38 []
52 []
76 []


In [104]:
[40]

163.40387

In [105]:
np.sum(cell_types[40])

150.99785

In [96]:
for i,ele in enumerate(cell_types):
    if ele.size!=7:
        print(i,ele.size)
        print(ele)

37 0
[]
38 0
[]
52 0
[]
74 0
[]


In [26]:
f["ticlDumper/simtrackstersCP"].keys()

['event',
 'NTracksters',
 'time',
 'timeBoundary',
 'timeError',
 'regressed_energy',
 'regressed_pt',
 'raw_energy',
 'raw_em_energy',
 'raw_pt',
 'raw_em_pt',
 'barycenter_x',
 'barycenter_y',
 'barycenter_z',
 'barycenter_eta',
 'barycenter_phi',
 'pdgID',
 'trackIdx',
 'trackTime',
 'boundaryX',
 'boundaryY',
 'boundaryZ',
 'boundaryEta',
 'boundaryPhi',
 'boundaryPx',
 'boundaryPy',
 'boundaryPz',
 'track_boundaryX',
 'track_boundaryY',
 'track_boundaryZ',
 'track_boundaryEta',
 'track_boundaryPhi',
 'track_boundaryPx',
 'track_boundaryPy',
 'track_boundaryPz',
 'EV1',
 'EV2',
 'EV3',
 'eVector0_x',
 'eVector0_y',
 'eVector0_z',
 'sigmaPCA1',
 'sigmaPCA2',
 'sigmaPCA3',
 'id_probabilities',
 'vertices_indexes',
 'vertices_x',
 'vertices_y',
 'vertices_z',
 'vertices_time',
 'vertices_timeErr',
 'vertices_energy',
 'vertices_correctedEnergy',
 'vertices_correctedEnergyUncertainty',
 'vertices_multiplicity']

In [30]:
keys = f["ticlDumper/simtrackstersCP"].keys()

for key in keys:
    if "energy" in key:
        print(key, f["ticlDumper/simtrackstersCP"][key].array())

regressed_energy [[67.9], [325], [447], [473], [47.7], ..., [152], [487], [23.9], [257], [566]]
raw_energy [[51.1], [338], [412], [429], [39.7], ..., [134], [402], [22.7], [217], [524]]
raw_em_energy [[13.4], [8.58], [373], [81.4], [0.389], ..., [9.66], [0.139], [0.478], [393]]
vertices_energy [[[5.14, 4.91, 3.85, 3.16, 1.9, ..., 0.0253, 0.0239, 0.0232, 0.0211]], ...]
